This project uses a collection of [Landlab](https://landlab.csdms.io/)
model runs. This notebook goes over a brief explanation of the model set
up and parameters and then downloading and processing of the model runs
from Zenodo.

# The Model

The model uses the following Landlab components:

-   [LinearDiffuser](https://landlab.csdms.io/generated/api/landlab.components.diffusion.diffusion.html#landlab.components.diffusion.diffusion.LinearDiffuser)

-   [FlowAccumulat](https://landlab.csdms.io/generated/api/landlab.components.flow_accum.flow_accumulator.html#landlab.components.flow_accum.flow_accumulator.FlowAccumulator)or

-   [FastscapeEroder](https://landlab.csdms.io/generated/api/landlab.components.stream_power.fastscape_stream_power.html#landlab.components.stream_power.fastscape_stream_power.FastscapeEroder)

    The model is implemented as subclass of the LandlabModel class to
    allow for batch runs to be orchestrated by the [Landlab
    Batcher](https://github.com/jrymart/landlab_batcher/tree/main)
    utility. The code of the model is as follows:

In [ ]:
from landlab.core import load_params
from landlab.components import LinearDiffuser, FlowAccumulator, FastscapeEroder
from model_base import LandlabModel
import numpy as np

class SimpleLem(LandlabModel):
    def __init__(self, params={}):
        """Initialize the Model"""
        super().__init__(params)

        if not ("topographic__elevation" in self.grid.at_node.keys()):
            self.grid.add_zeros("topographic__elevation", at="node")
        rng = np.random.default_rng(seed=int(params["seed"]))
        grid_noise= rng.random(self.grid.number_of_nodes)/10
        self.grid.at_node["topographic__elevation"] += grid_noise
        self.topo = self.grid.at_node["topographic__elevation"]

        self.uplift_rate = params["baselevel"]["uplift_rate"]
        self.diffuser = LinearDiffuser(
            self.grid,
            linear_diffusivity = params["diffuser"]["D"]
            )
        self.accumulator = FlowAccumulator(self.grid, flow_director="D8")
        self.eroder = FastscapeEroder(self.grid,
                                      K_sp=params["streampower"]["k"],
                                      m_sp=params["streampower"]["m"],
                                      n_sp=params["streampower"]["n"],
                                      threshold_sp=params["streampower"]["threshold"])


    def update(self, dt):
        """Advance the model by one time step of duration dt."""
        if self.current_time % 10000 == 0:
            print("Model %s on year %d" % (self.run_id, self.current_time))
        self.topo[self.grid.core_nodes] += self.uplift_rate * dt
        self.diffuser.run_one_step(dt)
        self.accumulator.run_one_step()
        self.eroder.run_one_step(dt)
        self.current_time += dt


The model is initialized with random noise as elevation, an uplift rate,
D8 flow router and accumulator and a diffusion and streampower erosion
component with parameters defiend by the study. The parameters of this
study are as follows:

~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–+

|  |  |  |  |  |  |  |  |  |  |
|----|----|----|----|----|----|----|----|----|----|
| Rows (cells) | Columns (cells) | Grid Spacing (m) | Runtime (years) | Timestep (years) | Uplift Rate () | Diffusivity ($\frac{m^2}{year}$) | Streampower K ($\frac{m^{0.4}}/year$) | Streampower m | Streampower n |

~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–+

|     |     |     |           |       |            |               |     |      |
|-----|-----|-----|-----------|-------|------------|---------------|-----|------|
| 300 | 100 | 5   | 3,000,000 | 0.001 | 0.005-0.02 | 0.00015-0.002 | 0.3 | 0.07 |

~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–~~—–+ This parameter set was run
with 10 different random seeds, and 30 linearly spaced values across the
parameter range for a dataset of 9000 model runs. The final Landlab
Grids of the model runs along with parameter information and associated
database for the Landlab batcher utility, are available on Zenodo:
<https://doi.org/10.5281/zenodo.15311644>.

# Downloading Model Topography

[Pooch](https://www.fatiando.org/pooch/latest/index.html) is used to
download the data files from Zenodo. If you have previously downloaded
the data pooch should not re-download.

In [ ]:
import pooch
from config import DATA_PATH, NPY_URL, NPY_HASH, DB_URL, DB_HASH, MODEL_DEM_DIR

DATA_PATH.mkdir(exist_ok=True, parents=True)
model_dem_paths = pooch.retrieve(url=NPY_URL,
                           known_hash=NPY_HASH,
                           path=DATA_PATH,
                           processor=pooch.Untar(extract_dir=MODEL_DEM_DIR))
db_path = pooch.retrieve(url=DB_URL,
                           known_hash=DB_HASH,
                           fname="model_runs.db",
                           path=DATA_PATH)


# Processing data for Flow Accumulation, Slope, and Curvature Rasters

We generate flow accumulation roasters for each model DEM for later
analysis

In [ ]:
import numpy as np
import os
from landlab import RasterModelGrid
from landlab.components import FlowAccumulator
from config import FLOW_METHOD, MODEL_ACC_PATH, MODEL_RESOLUTION, REPROCESS_DATA, MODEL_SLOPE_PATH, MODEL_CURV_PATH
from pathlib import Path

for dem_path in model_dem_paths:
    dem_path = Path(dem_path)
    dem_id = dem_path.stem
    slope_path = MODEL_SLOPE_PATH / f"{dem_id}.npy"
    curvature_path = MODEL_CURV_PATH / f"{dem_id}.npy"
    acc_path = MODEL_ACC_PATH / f"{dem_id}.npy"
    if slope_path.exists() and curvature_path.exists() and acc_path.exists() and not REPROCESS_DATA:
        continue
    dem_array = np.load(dem_path)
    if not acc_path.exists() or REPROCESS_DATA:
        grid = RasterModelGrid(dem_array.shape, MODEL_RESOLUTION)
        grid.add_field("node", "topographic__elevation", dem_array, units="m")
        accumulator = FlowAccumulator(grid, flow_director = FLOW_METHOD)
        accumulator.run_one_step()
        # this step is to ensure consistency in processing with real data
        flow_acc_array = grid.at_node["drainage_area"].reshape(grid.shape)
        np.save(acc_path, flow_acc_array)
    if not (slope_path.exists() or curvature_path.exists()) or REPROCESS_DATA:
        dz_dy, dz_dx = np.gradient(dem_array, MODEL_RESOLUTION)
    if not slope_path.exists() or REPROCESS_DATA:
        slope = np.sqrt(dz_dx**2+dz_dy**2)
        np.save(slope_path, slope)
    if not curvature_path.exists() or REPROCESS_DATA:
        dz_xy, dz_xx = np.gradient(dz_dx, MODEL_RESOLUTION)
        dz_yy, dz_yx = np.gradient(dz_dy, MODEL_RESOLUTION)
        laplacian = dz_xx+dz_yy
        np.save(curvature_path, laplacian)


# Data Statistics

We calculate some simple data statistics which will be used for
normalizing the data prior to training the neural networks. We calculate
statistics from the portion of the dataset used in training, so as not
to taint the dataset with statistical information from the testing set.
While our data is drawn from one distribution, this is beind done as a
best practice.

In [ ]:
import numpy as np
def get_array_statistics(array_paths, crop):
    array_total_sum = 0.0
    array_total_sum_sq = 0.0
    array_total_count = 0
    for path in array_paths:
        data_array=np.load(path)[crop:-crop,crop:-crop]
        array_total_sum += np.sum(data_array)
        array_total_sum_sq += np.sum(np.square(data_array))
        array_total_count += data_array.size
    array_mean = array_total_sum / array_total_count
    variance = (array_total_sum_sq / array_total_count) - np.square(array_mean)
    array_std = np.sqrt(variance)
    return {'inputs_mean': array_mean, 'inputs_std': array_std}

import sqlite3
import json
from config import SPLIT_BY_FIELD, TRAINING_FRACTION, PARAM_TABLE, RUN_ID_FIELD, MODEL_DEM_PATH, MODEL_ARRAY_CROP, LABEL_QUERY, OUTPUTS_TABLE, MODEL_STATS_PATH, DB_PATH


We use a parameter in the model parameter database (in this case) the
seed to split between train and test Datasets, so we need to connect to
and query the database for runs.

In [ ]:
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()
cursor.execute(f"SELECT DISTINCT \"{SPLIT_BY_FIELD}\" FROM {PARAM_TABLE}")
categories = [r[0] for r in cursor.fetchall()]
split = int((len(categories) * TRAINING_FRACTION))
train_categories = categories[:split]
train_filter = f"\"{SPLIT_BY_FIELD}\" IN ({', '.join([str(c) for c in train_categories])})"
cursor.execute(f"SELECT {RUN_ID_FIELD} FROM {PARAM_TABLE} WHERE {train_filter}")
train_run_ids = [r[0] for r in cursor.fetchall()]
# TODO don't need need to join, can import path dirm fix names also
train_dem_paths = [MODEL_DEM_PATH / f"{name}.npy" for name in train_run_ids]
train_acc_paths = [MODEL_ACC_PATH / f"{name}.npy" for name in train_run_ids]
train_slope_paths = [MODEL_SLOPE_PATH / f"{name}.npy" for name in train_run_ids]
train_curvature_paths = [MODEL_CURV_PATH / f"{name}.npy" for name in train_run_ids]
if not RECALCULATE_STATS:
    with open(MODEL_STATS_PATH, 'r') as f:
        statistics = json.load(f)
else:
    statistics = {}
data_types = {'dem': train_dem_paths, 'accumulation': train_acc_paths, 'slope': train_slope_paths, 'curvature': train_curvature_paths}
for data_type in data_types.keys():
    if RECALCULATE_STATS or data_type not in statistics:
        statistics[data_type] = get_array_statistics(data_types[data_type], MODEL_ARRAY_CROP)


Lastly, we need the statistics of the labels we will train the network
to infer.

In [ ]:
labels = []
limit = connection.getlimit(sqlite3.SQLITE_LIMIT_VARIABLE_NUMBER)
for i in range(0, len(train_run_ids), limit):
    current_chunk_runs = train_run_ids[i:i+limit]
    # placeholders are a safe way to programatically construct an SQL query
    placeholders = ', '.join(['?']*len(current_chunk_runs))
    cursor.execute(f"{LABEL_QUERY} WHERE {RUN_ID_FIELD} IN ({placeholders})", current_chunk_runs)
    labels += [l[0] for l in cursor.fetchall()]
labels = np.array(labels, dtype=np.float64)
statistics['labels'] = {'labels_mean': np.mean(labels),
                       'labels_std' : np.std(labels)}
with open(MODEL_STATS_PATH, 'w') as f:
    json.dump(statistics, f)
